# Limpieza y preprocesamiento de datos con pandas

## 1. Cargamos el dataset

In [2]:
import pandas as pd
# Cargamos el dataset
df = pd.read_csv('../data/dataset_original.csv', encoding='ISO-8859-1')
# Mostramos las primeras filas
df.head(4)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 2. Exploración inicial de los datos

### Dimensiones del dataset

In [21]:
print("\nDimensiones del dataset:", df.shape)


Dimensiones del dataset: (541909, 8)


### Información sobre columnas y tipos de datos

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### Descripción estadística de los datos numéricos

In [23]:
# Descripción estadística de los datos numéricos
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


### Verificación de valores nulos

In [24]:
print(df.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


## 3. Limpieza y procesado de los datos

### Procesar filas con valores nulos
En el punto 2 hemos visto como las columnas con valores nulos son "CustomerID" y "Description". Así que procesamos esos

In [25]:
import random

df['Description'] = df.groupby('StockCode')['Description'].transform(lambda x: x.ffill().bfill())
df['Description'] = df['Description'].fillna('No description')

def generate_unique_customer_id(current_ids):
  while True:
    new_id = random.randint(10000, 99999)
    if new_id not in current_ids:
      return new_id
        
existing_ids = set(df['CustomerID'].dropna().astype(int))

df['CustomerID'] = df['CustomerID'].apply(
  lambda x: generate_unique_customer_id(existing_ids) if pd.isnull(x) else x
)

df.isnull().sum()

C:\Users\Wiler\AppData\Local\Temp\ipykernel_5872\1733775019.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Description'] = df.groupby('StockCode')['Description'].transform(lambda x: x.ffill().bfill())


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

### Manejo de Outliers

In [26]:
# TODO: ECHARLE UN OJO A ESTO PARA SABER SI TIENE SENTIDO
df['Quantity'] = df['Quantity'].abs()
df['UnitPrice'] = df['UnitPrice'].abs()

print("Valores negativos en 'Quantity':", (df['Quantity'] < 0).sum())
print("Valores negativos en 'UnitPrice':", (df['UnitPrice'] < 0).sum())

Valores negativos en 'Quantity': 0
Valores negativos en 'UnitPrice': 0


### Eliminar duplicados

In [27]:
df = df.drop_duplicates()
print("\nDespués de eliminar duplicados, dimensiones del dataset:", df.shape)


Después de eliminar duplicados, dimensiones del dataset: (536684, 8)


## 4. Completando los datos 

### Calcular las ventas totales por cada transacción

In [28]:
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

### Dividir la fecha en varias columnas

In [29]:
# Convertimos `InvoiceDate` a formato datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Creamos columnas para el año, mes, día, día de la semana y semana del año
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['Season'] = df['Month'].apply(lambda x: 'Winter' if x in [12, 1, 2] else
                                        ('Spring' if x in [3, 4, 5] else
                                         ('Summer' if x in [6, 7, 8] else 'Fall')))

## 5. Exploración final de los datos

### Dimensiones del dataset

In [30]:
print("\nDimensiones del dataset:", df.shape)


Dimensiones del dataset: (536684, 14)


### Información sobre columnas y tipos de datos

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 536684 entries, 0 to 541908
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536684 non-null  object        
 1   StockCode    536684 non-null  object        
 2   Description  536684 non-null  object        
 3   Quantity     536684 non-null  int64         
 4   InvoiceDate  536684 non-null  datetime64[ns]
 5   UnitPrice    536684 non-null  float64       
 6   CustomerID   536684 non-null  float64       
 7   Country      536684 non-null  object        
 8   TotalSales   536684 non-null  float64       
 9   Year         536684 non-null  int32         
 10  Month        536684 non-null  int32         
 11  Day          536684 non-null  int32         
 12  DayOfWeek    536684 non-null  int32         
 13  Season       536684 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(4), int64(1), object(5)
memory usage: 53.2+ MB


### Descripción estadística de los datos numéricos

In [32]:
# Descripción estadística de los datos numéricos
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalSales,Year,Month,Day,DayOfWeek
count,536684.000000,536684,536684.000000,536684.000000,536684.000000,536684.000000,536684.000000,536684.000000,536684.000000
mean,11.418132,2011-07-04 08:52:38.508135936,4.714935,25792.933903,21.537384,2010.921770,7.544727,15.024836,2.423892
min,1.000000,2010-12-01 08:26:00,0.000000,10001.000000,0.000000,2010.000000,1.000000,1.000000,0.000000
25%,1.000000,2011-03-28 10:45:30,1.250000,14292.000000,3.750000,2011.000000,5.000000,7.000000,1.000000
50%,3.000000,2011-07-19 14:04:00,2.080000,16161.000000,9.900000,2011.000000,8.000000,15.000000,2.000000
75%,11.000000,2011-10-18 17:05:00,4.130000,18180.000000,17.700000,2011.000000,11.000000,22.000000,4.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,99997.000000,168469.600000,2011.000000,12.000000,31.000000,6.000000
std,219.035146,NaN,97.225265,22104.832881,380.463163,0.268534,3.508713,8.663352,1.839883


### Verificación de valores nulos

In [33]:
print(df.isnull().sum())

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalSales     0
Year           0
Month          0
Day            0
DayOfWeek      0
Season         0
dtype: int64


## 6. Guardamos el dataset limpio en CSV

In [34]:
# Guardar dataset limpio
df.to_csv('../data/dataset_limpio.csv', index=False)

## 7. Creamos y guardamos los datasets para el modelo

In [38]:

date_to_split_data = pd.to_datetime("10/08/2011 00:00", format="%m/%d/%Y %H:%M")

df_to_train_part = df[df['InvoiceDate']<=date_to_split_data]
df_to_test_part = df[df['InvoiceDate'] > date_to_split_data]


df_to_train_grouped = df_to_train_part.groupby(['Year', 'Month', 'Day', 'DayOfWeek', 'Season'])['TotalSales'].sum().reset_index()
df_to_test_grouped  = df_to_test_part.groupby(['Year', 'Month', 'Day', 'DayOfWeek', 'Season'])['TotalSales'].sum().reset_index()

# TODO Calcular dummies aqui o NO????? HACE FALTA??
#df_to_train = pd.get_dummies(df_to_train_grouped, columns=['Season'], drop_first=True)
#df_to_test = pd.get_dummies(df_to_test_grouped, columns=['Season'], drop_first=True)

# ! Si decidimos poner los dummies aqui, entonces hay que cambiar esto
df_to_train_grouped.to_csv('../data/dataset_de_entrenamiento.csv')
df_to_test_grouped.to_csv('../data/dataset_de_testing.csv')



